# CHAPTER 11. Smarter Email Marketing with the Markov Model

구매이력을 기반으로 해서 고객 대상의 "스마트한 이메일 마켓팅"할 날짜를 예측해보자.

고객의 마지막 구매 날짜가 랜덤(설명) 변수로 주어지고, Markov Model은 이전 구매이력 데이터를 가지고 구매 날짜에 대한 분포를 구함.

## Markov Chains에 대한 설명

> 유한한 상태들의 집합을  \\(  S = ( S_1, S_2, S_3, ... )  \\)  라고 정의

> 우리는 다음과 같이 확률을 구할 수 있음.

![](chap11_01.jpg)

> Markov’s first-order 가정은 다음과 같음.

> t + 1 시간의 상태는 오직 t 시간의 상태에 의존함.

![](chap11_02.jpg)

> Markov’s second-order 가정은 다음과 같음.

![](chap11_03.jpg)

> 이것을 계속 확장하여  Markov 가정을  결합확률로 표현할 수 있음.

![](chap11_04.jpg)

> 예를 들어서 설명하면,  4가지 날씨 상태( sunny, cloudy, rainy, foggy )가 있다고 가정하고, 날씨패턴은 아래와 같음.

![](chap11_05.jpg)

> 오늘 날씨가 sunny가 주어진다면, 내일은 cloudy 이고 그 다음날은 foggy일 확률은 ?

![](chap11_06.jpg)


> 오늘 날씨가 foggy가 주어진다면, 오늘부터 이틀후에 rainy일 확률은?

![](chap11_07.jpg)

이번장의 목표는  모델( = transition table )을 만드는 것임.

## Spark Solution 

### Input Format

```
<customerID><,><transactionID><,><purchaseDate><,><amount>


V31E55G4FI,1381872898,2013-01-01,123
301UNH7I2F,1381872899,2013-01-01,148
PP2KVIR4LD,1381872900,2013-01-01,163
AC57MM3WNV,1381872901,2013-01-01,188
BN020INHUM,1381872902,2013-01-01,116
UP8R2SOR77,1381872903,2013-01-01,183
VD91210MGH,1381872904,2013-01-01,204
COI4OXHET1,1381872905,2013-01-01,78
76S34ZE89C,1381872906,2013-01-01,105
6K3SNF2EG1,1381872907,2013-01-01,214
```

### High-Level Steps

1) 입력 파라메터에 대한 처리 :  구매이력 데이터 파일의 경로를 입력값으로 필요하지만 여기서는 하드 코딩함.

2) Spark Context을 만들고 입력데이터로부터 RDD < String > 을 생성

3) RDD < String > 을 RDD < K, V > 로 변환.  K는 customerID,  V는 tuple < purchaseData, amount> 임.

4) customerID을 기준으로 구매이력을 그룹핑하며 RDD < K2 , V2 >결과로 나옴.  K2는 customerID,  V2는 Iterable < Tuple < purchaseDate, Amount > > 임.

5) Markov state sequence을 만듬 :  State1, State2, ..., StateN 
   - mapValues()으로 customerID에 해당하는 V2 값들을 가지고 와서 변환 작업을 수행
   
6) Markov state transition을 생성
  - Input :  PairRDD < K4, V4 > pairs
  - Output : 
![](chap11_08.jpg)

7) 최종 결과를 저장

## Spark Program